In [ ]:
# cafe data cleaning and vizulizing project
import numpy as np;
import pandas as pd;

import matplotlib.pyplot as plt;
import seaborn as see;


In [ ]:
# get dataset 
cafe=pd.read_csv(r"C:\Users\Aniket\OneDrive\Desktop\python_for_DS\cafe_project\dirty_cafe_sales (1).csv")
print(cafe.head());

  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  


In [ ]:
# anaylising null and dirty vALUES IN THE EACH COLUMN 
print(cafe.isna().sum())
colum=['Item','Quantity','Price Per Unit','Total Spent','Payment Method','Location']
for col in colum:
    print("------------- value count in ",col,"-------------\n\n")
    print(cafe[col].value_counts(dropna=False))
print("these are the all values in the every column in table")    


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64
------------- value count in  Item -------------


Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
NaN          333
ERROR        292
Name: count, dtype: int64
------------- value count in  Quantity -------------


Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
NaN         138
Name: count, dtype: int64
------------- value count in  Price Per Unit -------------


Price Per Unit
3.0        2429
4.0        2331
2.0        1227
5.0        1204
1.0        1143
1.5        1133
ERROR       190
NaN         179
UNKNOWN     164
Name: count, dtype: int64
------------- value count in  Total Spent ------------

In [ ]:
# marking NaN where Unknown and error exist 
for col in colum:
  cafe[col]=cafe[col].replace(['ERROR','UNKNOWN'],np.nan);
print(cafe.isna().sum())

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     159
dtype: int64


In [ ]:
#type conversion of numeric column 
cafe['Quantity']=pd.to_numeric(cafe['Quantity'], errors='coerce').astype('Int64')
cafe['Price Per Unit']=pd.to_numeric(cafe['Price Per Unit'],errors='coerce')
cafe['Total Spent']=pd.to_numeric(cafe['Total Spent'],errors='coerce')
print(cafe.dtypes)
print(cafe.isna().sum())


Transaction ID       object
Item                 object
Quantity              Int64
Price Per Unit      float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
dtype: object
Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     159
dtype: int64


In [ ]:
# heandling the null values in COLUMN
price_item_map = (
    cafe
    .dropna(subset=['Item'])
    .groupby('Price Per Unit')['Item']
    .agg(lambda x: x.mode()[0])
)

cafe['Item']=cafe['Item'].fillna(cafe["Price Per Unit"].map(price_item_map))


cafe["Item"].fillna(cafe["Item"].mode()[0],inplace=True)
print("null values in item table ",cafe["Item"].isna().sum())

null values in item table  0


In [ ]:
# price per itam filled 
price_fill=(cafe.dropna(subset=['Price Per Unit']).groupby('Item')['Price Per Unit'].agg(lambda x:x.mode()[0]))
cafe["Price Per Unit"]=cafe["Price Per Unit"].fillna(cafe["Item"].map(price_fill))
print("null value in price per unit table",cafe["Item"].isna().sum())

null value in price per unit table 0


In [ ]:

#  quentity column fill
cafe["Quantity"] = cafe["Quantity"].fillna(
    (cafe["Total Spent"] / cafe["Price Per Unit"]).round()
)
cafe["Quantity"]=cafe["Quantity"].fillna(cafe["Quantity"].mode()[0])
print("values are null in the quantity table",cafe["Quantity"].isna().sum())

values are null in the quantity table 0


In [ ]:
   # total spent filled
cafe["Total Spent"]=cafe["Total Spent"].fillna(cafe["Quantity"]*cafe["Price Per Unit"].round()) 
print("null values in Total spent ",cafe["Total Spent"].isna().sum())


print(cafe["Location"].value_counts(dropna=False))
print(cafe["Payment Method"].value_counts(dropna=False))

null values in Total spent  0
Location
NaN         3961
Takeaway    3022
In-store    3017
Name: count, dtype: int64
Payment Method
NaN               3178
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64


In [ ]:
# payment method fill by Item, orderd 

pay_filler = (
    cafe
    .dropna(subset=["Item", "Payment Method"])
    .groupby("Item")["Payment Method"]
    .agg(lambda x: x.mode()[0])
)
cafe["Payment Method"] = cafe["Payment Method"].fillna(
    cafe["Item"].map(pay_filler)
)
print(pay_filler)
print(cafe["Payment Method"].value_counts(dropna=False))

Item
Cake                  Cash
Coffee      Digital Wallet
Cookie      Digital Wallet
Juice       Digital Wallet
Salad                 Cash
Sandwich       Credit Card
Smoothie    Digital Wallet
Tea         Digital Wallet
Name: Payment Method, dtype: object
Payment Method
Digital Wallet    4306
Cash              2996
Credit Card       2698
Name: count, dtype: int64


In [ ]:

# location fill by order typee 
loc_filler = (
    cafe
    .dropna(subset=["Item", "Location"])
    .groupby("Item")["Location"]
    .agg(lambda x: x.mode()[0])
)
cafe['Location']=cafe["Location"].fillna(cafe["Item"].map(loc_filler))
print(loc_filler)
print(cafe["Location"].value_counts(dropna=False))

Item
Cake        Takeaway
Coffee      Takeaway
Cookie      Takeaway
Juice       In-store
Salad       In-store
Sandwich    In-store
Smoothie    In-store
Tea         In-store
Name: Location, dtype: object
Location
In-store    5528
Takeaway    4472
Name: count, dtype: int64


In [ ]:
# date filled 
cafe["Transaction Date"] = pd.to_datetime(
    cafe["Transaction Date"],
    errors="coerce"
)
cafe["Transaction Date"].fillna(method="ffill", inplace=True)


C:\Users\Aniket\AppData\Local\Temp\ipykernel_6604\265936857.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cafe["Transaction Date"].fillna(method="ffill", inplace=True)


In [ ]:
# final chack for data clean or not 
print(cafe.isna().sum())
cafe_clean = cafe.copy()

cafe_clean.to_csv("cafe_cleaned.csv", index=False)


Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64
